# Churn Prediction Model

**Churn rate** is a business term describing the rate at which customers leave or cease paying for a product or service. It's a critical figure in many businesses, as it's often the case that acquiring new customers is a lot more costly than retaining existing ones (in some cases, five to twenty times more expensive).

### External requirements
This prediction model was created using Python 2.7.12 (64-bit) through the Anaconda 4.1.1 distribution. These are the installation steps:

1. Download [Anaconda](https://www.continuum.io/downloads) for the operating system and architecture you are using. *Due to the large amount of data processing required for this model, the system must have at least 5GB of RAM meaning that the architecture must be 64-bit and not 32-bit.*
2. Run installer and complete installation.
3. Visit your system's `PATH` and make sure there are references to `/Anaconda`, `/Anaconda/Scripts` and `/Anaconda/Library/bin`. *Any changes to the `PATH` are reflected upon logout or restart depending on the system.*

All libraries except for `pydotplus` already come with Anaconda. You may install libraries through Terminal or Command Prompt issuing the command:<br/>`pip install pydotplus`.

Next, we need to import some necessary libraries for our project:
* [`numpy`](http://www.numpy.org/): Allows lists to be manipulated as arrays
* [`pandas`](http://pandas.pydata.org/): Allows arrays to be manipulated as data frames
* [`time`](https://docs.python.org/2/library/time.html): Allows to measure time
* [`warnings`](https://docs.python.org/2/library/warnings.html): Allows to control the warnings of the interpreter
* [`pydotplus`](https://pypi.python.org/pypi/pydotplus): Allows to convert graph objects to images and documents

In [1]:
import numpy
import pandas
import time
import warnings
import pydotplus

In [2]:
%matplotlib inline
import lifelines as ll
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from IPython.display import HTML
import matplotlib.pyplot as plt
import plotly.plotly as py
import plotly.tools as tls   
from plotly.graph_objs import *
from pylab import rcParams
py.sign_in('carlosgl87', 'k5copjs82g')

Next, we import some packages from the `scikit-learn`, `IPython` and `matplotlib` libraries:

#### Pre-processing
* [`preprocessing`](http://scikit-learn.org/stable/modules/preprocessing.html): Allows to transform and normalize data frames

#### Cross-Validation
* [`ShuffleSplit`](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.train_test_split.html): Allows to perform cross-validation split of training and testing sets using random permutations iteratively.

#### Classifiers
* [`SVC`](http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVC.html): Allows to instance Support Vector Machine Classifiers
* [`GaussianNB`](http://scikit-learn.org/stable/modules/generated/sklearn.naive_bayes.GaussianNB.html): Allows to instance Gaussian Naïve Bayes Classifiers
* [`DecisionTreeClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html): Allows to instance Decision Tree Classifiers
* [`RandomForestClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html): Allows to instance Random Forest Classifiers
* [`GradientBoostingClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.GradientBoostingClassifier.html): Allows to instance Gradient Boosting Classifiers
* [`KNeighborsClassifier`](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html): Allows to instance K-Nearest Neighbors Classifiers

#### Searchers
* [`GridSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.GridSearchCV.html): Allows to perform grid searches over specified parameter values for an estimator
* [`RandomizedSearchCV`](http://scikit-learn.org/stable/modules/generated/sklearn.grid_search.RandomizedSearchCV.html): Allows to perform a randomized search on hyper paremeters

#### Evaluators
* [`f1_score`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.f1_score.html): Allows to score model results using the F1 score using the weighted average of the precision and recall
* [`accuracy_score`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html): Allows to score model results using the harmonic mean between precision and recall
* [`confusion_matrix`](http://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html#sklearn.metrics.confusion_matrix): Allows to create a confusion matrix to compare true/false positives and true/false negatives, recall and precision

#### Visualizers
* [`export_graphviz`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.export_graphviz.html): Allows to use the Graphviz module to create graphs objects from model results
* [`display`](https://ipython.org/ipython-doc/3/api/generated/IPython.display.html): Allows to display data frames as tables
* [`pyplot`](http://matplotlib.org/api/pyplot_api.html): Allows to create some plot in-line with the notebook

#### Other modules
* [`StringIO`](https://github.com/scikit-learn/scikit-learn/blob/master/sklearn/externals/six.py): Allows to read and write file names as strings on the file system

In [3]:
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.externals.six import StringIO
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cross_validation import ShuffleSplit
from sklearn import preprocessing
from IPython.display import display
from matplotlib import pyplot

### Loading data
Now that we have imported the necessary libraries and packages, we are ready to load the data:

In [4]:
path = 'D:\\cgamero'
data = pandas.read_csv(path + "\\churn_data_total_sinNA_1.csv")

C:\Anaconda2\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning:

Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.



Now we need to determine how many clients we have information on, and learn about the churn rate among these clients:

In [5]:
churn_size = data.ABANDONO_A[data.ABANDONO_A == 1].count()
churn_rate = (float(churn_size) / float(churn_size + data.ABANDONO_A[data.ABANDONO_A == 0].count()))*100

print "Total number of clients: {}".format(data.shape[0])
print "\nNumber of features: {}".format(data.shape[1] - 1)
print "Number of clients who churn: {}".format(churn_size)
print "Number of clients who stay: {}".format(data.ABANDONO_A[data.ABANDONO_A == 0].count())
print "\nChurn rate of the Clients: {:.3f}%".format(churn_rate)

Total number of clients: 233227

Number of features: 273
Number of clients who churn: 104598
Number of clients who stay: 128629

Churn rate of the Clients: 44.848%


### Preparing the data

Now we need to take a look at the variables (features) that we have in our data:

In [6]:
print list(data.columns)

['Unnamed: 0', 'COLOR', 'MODELO_VEHICULO', 'USO_VEHICULO_INICIAL', 'USO_VEHICULO_FINAL', 'USO_VEHICULO_CAMBIO', 'LIMA_PROVINCIAS', 'NSE', 'SEGMENTO', 'CAMBIO_CARRO', 'SUMA_ASEGURADA_MEDIA', 'SUMA_ASEGURADA_MEDIANA', 'SUMA_ASEGURADA_SUMA', 'PRIMA_TOTAL', 'PRIMA_INICIAL', 'PRIMA_FINAL', 'CAMBIO_PRIMA', 'ABANDONO_NA', 'ABANDONO_A', 'ANULADO_NA', 'NO_RENOVACION_NA', 'CANAL_DISTRIBUCION', 'CORREDOR_MARSH', 'CORREDOR_CONSEJEROS_CORREDORES_SEGUROS', 'CORREDOR_CORREDORES_SEGUROS_FALABELLA', 'CORREDOR_PRIMERA_CORREDORES_SEGURO', 'CORREDOR_MARIATEGUI', 'CORREDOR_LA_UNION', 'CORREDOR_CARLOS_UGAZ', 'CORREDOR_GERENCIA_RIESGOS', 'CORREDOR_GABEL', 'CORREDOR_CONTACTO_CORREDORES', 'CORREDOR_MN_ASOC', 'CORREDOR_AMERICA_BROKERS', 'CORREDOR_FBA_CORREDORES', 'CORREDOR_OTROS', 'MONTO_SINIESTRO_MEDIA', 'MONTO_SINIESTRO_MEDIANA', 'MONTO_SINIESTRO_SUMA', 'CAUSA_SINIESTROS_ATROPELLO', 'CAUSA_SINIESTROS_CHOQUE_ESTACIONADO', 'CAUSA_SINIESTROS_CHOQUE_FUGA', 'CAUSA_SINIESTROS_CHOQUE_CIRCULANDO', 'CAUSA_SINIESTROS_D

We have identified that the feature `tiempo` is displayed in days. We add three more features that could possibly help us understand time better:

1. `tiempo_anos`: years the client has been a client
2. `tiempo_sem`: semesters the client has been a client
3. `tiempo_trim`: quarters the client has been a client

In [7]:
data['tiempo_anos'] = ((data['tiempo'] / 365).astype('int') + 1)
data['tiempo_sem'] = ((data['tiempo'] / 182.5).astype('int') + 1)
data['tiempo_trim'] = ((data['tiempo'] / 91.25).astype('int') + 1)

Now we remove some features we don't need:

In [21]:
churn_data_1 = data[['tiempo','ABANDONO_A',
    'LIMA_PROVINCIAS_CONO', 'LIMA_PROVINCIAS_LIMA TRADICIONAL', 'LIMA_PROVINCIAS_PROVINCIA',
    'NSE_A', 'NSE_A1', 'NSE_A2', 'NSE_B1', 'NSE_B2', 'NSE_C1', 'NSE_C2', 'NSE_D1', 'NSE_D2', 'NSE_E',
    'SEGMENTO_MASIVO', 'SEGMENTO_ORO 1', 'SEGMENTO_ORO 2', 'SEGMENTO_ORO 3', 'SEGMENTO_PLATINO 1', 'SEGMENTO_PLATINO 2',
    'n_EDAD',
    'MARCA_AUDI', 'MARCA_BAJAJ', 'MARCA_BMW', 'MARCA_CHERY', 'MARCA_CHEVROLET', 'MARCA_CITROEN', 'MARCA_DAIHATSU', 
    'MARCA_DODGE', 'MARCA_FORD', 'MARCA_GREAT WALL', 'MARCA_HONDA', 'MARCA_HYUNDAI', 'MARCA_JAC', 'MARCA_JEEP', 'MARCA_KIA', 
    'MARCA_MAZDA', 'MARCA_MERCEDES', 'MARCA_MITSUBISHI', 'MARCA_NISSAN', 'MARCA_OTROS', 'MARCA_RENAULT', 'MARCA_SEAT', 
    'MARCA_SSANGYONG', 'MARCA_SUBARU', 'MARCA_SUZUKI', 'MARCA_TOYOTA', 'MARCA_VOLKSWAGEN', 'MARCA_VOLVO',
    'USO_VEHICULO_INICIAL_CARGA', 'USO_VEHICULO_INICIAL_COMERCIAL', 'USO_VEHICULO_INICIAL_OTRO', 
    'USO_VEHICULO_INICIAL_PARTICULAR', 'USO_VEHICULO_INICIAL_PUBLICO', 'USO_VEHICULO_INICIAL_TAXI URBANO', 
    'USO_VEHICULO_INICIAL_TRANSPORTE DE PERSONAL', 'USO_VEHICULO_INICIAL_VEHICULO DE ALQUILER',
    'n_TIEMPO_FABRICACION',
    'PRIMA_INICIAL',
    'CANAL_CORREDOR', 'CANAL_DIRECTO', 'CANAL_FUERZA DE VENTAS', 'CANAL_NO TRADICIONAL',
    'CORREDOR_MARSH', 'CORREDOR_CONSEJEROS_CORREDORES_SEGUROS', 'CORREDOR_CORREDORES_SEGUROS_FALABELLA', 
    'CORREDOR_PRIMERA_CORREDORES_SEGURO', 'CORREDOR_MARIATEGUI', 'CORREDOR_LA_UNION', 'CORREDOR_CARLOS_UGAZ', 
    'CORREDOR_GERENCIA_RIESGOS', 'CORREDOR_GABEL', 'CORREDOR_CONTACTO_CORREDORES', 'CORREDOR_MN_ASOC', 
    'CORREDOR_AMERICA_BROKERS', 'CORREDOR_FBA_CORREDORES', 'CORREDOR_OTROS',
    'MONTO_SINIESTRO_SUMA', 
    'RECLAMO', 'DISCONFORMIDADES',
    'n_NUMERO_SINIESTROS_FINAL',
    'n_NUM_RIESGOS',
    'n_NUM_PRODUCTOS',
    'n_NUMERO_SINIESTROS_TOTAL']]

In [20]:
def num_missing(x):
    return sum(x.isnull())
df1_nulls = churn_data_1.apply(num_missing, axis=0)
print df1_nulls[df1_nulls > 0].sort_values(ascending=False)

PERDIDA_INICIAL       59355
DUDOSO_INICIAL        59355
DEFICIENTE_INICIAL    59355
CPP_INICIAL           59355
OK_INICIAL            59355
dtype: int64


In [ ]:
churn_data_1 = data.drop([
        'Unnamed: 0',
        'CAMBIO_CARRO',
        'SUMA_ASEGURADA_MEDIANA',
        'SUMA_ASEGURADA_SUMA',
        'PRIMA_TOTAL',
        'MONTO_SINIESTRO_MEDIANA',
        'MONTO_SINIESTRO_SUMA',
        'ANO_FABRICACION_2000-2007',
        'ANO_FABRICACION_2008',
        'ANO_FABRICACION_2009',
        'ANO_FABRICACION_2010',
        'ANO_FABRICACION_2011',
        'ANO_FABRICACION_2012',
        'ANO_FABRICACION_2013',
        'ANO_FABRICACION_2014',
        'ANO_FABRICACION_2015',
        'ANO_FABRICACION_2016',
        'ANO_FABRICACION_>2000',
        'ANO_FABRICACION_nan',
        'COLOR_nan',
        'MARCA_nan',
        'MODELO_nan',
        'TIPO_VEHICULO_nan',
        'EDAD_nan',
        'TIPO_VEHICULO_VAN',
        'LIMA_PROVINCIAS_nan',
        'NSE_nan',
        'SEGMENTO_MASIVO',
        'SEGMENTO_ORO 1',
        'SEGMENTO_ORO 2',
        'SEGMENTO_ORO 3',
        'SEGMENTO_PLATINO 1',
        'SEGMENTO_PLATINO 2',
        'SEGMENTO_nan',
        'TIPO_PLAN_nan',
        'PLAN_BASICO',
        'PLAN_COLECTIVO',
        'PLAN_CORPORATIVO',
        'PLAN_MODULAR',
        'PLAN_NO_TRADICIONAL',
        'PLAN_PREMIER',
        'PLAN_PROVINCIA',
        'TIEMPO_FABRICACION_nan',
        'NUMERO_SINIESTROS_TOTAL_0',
        'NUMERO_SINIESTROS_TOTAL_1',
        'NUMERO_SINIESTROS_TOTAL_2',
        'NUMERO_SINIESTROS_TOTAL_3 - 4',
        'NUMERO_SINIESTROS_TOTAL_> 5',
        'NUMERO_CERTIFICADOS_1',
        'NUMERO_CERTIFICADOS_2',
        'NUMERO_CERTIFICADOS_3',
        'NUMERO_CERTIFICADOS_4',
        'NUMERO_CERTIFICADOS_>4',
        'NUMERO_SINIESTROS_FINAL_0',
        'NUMERO_SINIESTROS_FINAL_1',
        'NUMERO_SINIESTROS_FINAL_2',
        'NUMERO_SINIESTROS_FINAL_3 - 4',
        'NUMERO_SINIESTROS_FINAL_> 5',
        'NUM_RIESGOS_0',
        'NUM_RIESGOS_1',
        'NUM_RIESGOS_2',
        'NUM_RIESGOS_>3',
        'NUM_PRODUCTOS_0', 
        'NUM_PRODUCTOS_1',
        'NUM_PRODUCTOS_2',
        'NUM_PRODUCTOS_3',
        'NUM_PRODUCTOS_>4',
        'EDAD_30-35',
        'EDAD_35-40',
        'EDAD_40-50', 
        'EDAD_50-60',
        'EDAD_<30',
        'EDAD_>60',
        'TIEMPO_FABRICACION_0',
        'TIEMPO_FABRICACION_1',
        'TIEMPO_FABRICACION_2',
        'TIEMPO_FABRICACION_3',
        'TIEMPO_FABRICACION_4',
        'TIEMPO_FABRICACION_5',
        'TIEMPO_FABRICACION_6 - 10',
        'TIEMPO_FABRICACION_>10',
        'tiempo_anos',
        'tiempo_sem',
        'tiempo_trim',
        'ABANDONO_NA',
        'ANULADO_NA', 
        'NO_RENOVACION_NA',
        'n_ANO_FABRICACION',
        'n_NUMERO_CERTIFICADOS',
    ], axis=1)

Now we separate our features and target columns as well as our labels:

In [22]:
churn_labels = churn_data_1.columns.tolist()
feature_cols = churn_data_1.drop(['ABANDONO_A'], axis=1).columns.tolist()
target_col = ['ABANDONO_A']
class_names = ['STAY', 'CHURN']

print "Features: {}".format(feature_cols)
print "\nTarget: {}".format(target_col)

Features: ['tiempo', 'LIMA_PROVINCIAS_CONO', 'LIMA_PROVINCIAS_LIMA TRADICIONAL', 'LIMA_PROVINCIAS_PROVINCIA', 'NSE_A', 'NSE_A1', 'NSE_A2', 'NSE_B1', 'NSE_B2', 'NSE_C1', 'NSE_C2', 'NSE_D1', 'NSE_D2', 'NSE_E', 'SEGMENTO_MASIVO', 'SEGMENTO_ORO 1', 'SEGMENTO_ORO 2', 'SEGMENTO_ORO 3', 'SEGMENTO_PLATINO 1', 'SEGMENTO_PLATINO 2', 'n_EDAD', 'MARCA_AUDI', 'MARCA_BAJAJ', 'MARCA_BMW', 'MARCA_CHERY', 'MARCA_CHEVROLET', 'MARCA_CITROEN', 'MARCA_DAIHATSU', 'MARCA_DODGE', 'MARCA_FORD', 'MARCA_GREAT WALL', 'MARCA_HONDA', 'MARCA_HYUNDAI', 'MARCA_JAC', 'MARCA_JEEP', 'MARCA_KIA', 'MARCA_MAZDA', 'MARCA_MERCEDES', 'MARCA_MITSUBISHI', 'MARCA_NISSAN', 'MARCA_OTROS', 'MARCA_RENAULT', 'MARCA_SEAT', 'MARCA_SSANGYONG', 'MARCA_SUBARU', 'MARCA_SUZUKI', 'MARCA_TOYOTA', 'MARCA_VOLKSWAGEN', 'MARCA_VOLVO', 'USO_VEHICULO_INICIAL_CARGA', 'USO_VEHICULO_INICIAL_COMERCIAL', 'USO_VEHICULO_INICIAL_OTRO', 'USO_VEHICULO_INICIAL_PARTICULAR', 'USO_VEHICULO_INICIAL_PUBLICO', 'USO_VEHICULO_INICIAL_TAXI URBANO', 'USO_VEHICULO_INICIA

### Normalization and cross-validation

We need to make sure our data is given the same importance across features. To do this we normalize our values using the [`scale()`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.scale.html) function provided by the [`preprocessing`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing) package to create the `churn_norm` data frame. Then we slice the features we need for `X_all` and the target feature for `y_all`:

In [23]:
#data_norm = pandas.DataFrame(preprocessing.scale(churn_data_1.values), columns=churn_labels)

X_all = churn_data_1[feature_cols]
y_all = churn_data_1[target_col]

We need to split our data into training and testing sets. The models first train using some part of the data (training set) and then once some rules for classification have been established, test using another part of the data (testing set). 

The cross validation function we are using is called [`ShuffleSplit()`](http://scikit-learn.org/stable/modules/generated/sklearn.cross_validation.ShuffleSplit.html#sklearn.cross_validation.ShuffleSplit) from the [`cross_validation`](http://scikit-learn.org/stable/modules/classes.html#module-sklearn.cross_validation) package. This method performs a random permutation cross-validation iteratively for a specified number of iterations and partition size. We are using 100 iterations and a 30-70 test-training partitioning.

In [24]:
X = numpy.array(X_all)
y = numpy.array(y_all)

for i, j in ShuffleSplit(len(X), n_iter=100, test_size=.3):
    X_train, X_test = X[i], X[j]
    y_train, y_test = y[i], y[j]

print "The training set has {} samples.".format(X_train.shape[0])
print "The testing set has {} samples.".format(X_test.shape[0])

X_train, X_test = pandas.DataFrame(X_train), pandas.DataFrame(X_test)
y_train, y_test = pandas.DataFrame(y_train), pandas.DataFrame(y_test)

The training set has 163258 samples.
The testing set has 69969 samples.


### Comparing different models
Now that the data is ready and cross-validated, we can start to compare different models:

In [27]:
warnings.filterwarnings('ignore')

DTC = DecisionTreeClassifier()
# SVC = SVC()
GNB = GaussianNB()
RFC = RandomForestClassifier()
ETC = ExtraTreesClassifier()
KNN = KNeighborsClassifier()
GBC = GradientBoostingClassifier()

for classifier in [DTC, GNB, RFC, ETC, KNN, GBC]:
    classifier.fit(X_train, y_train)
    print classifier
    print '\nTraining set accuracy: {:.3f}%'.format(accuracy_score(y_train.values, classifier.predict(X_train))*100)
    print 'Test set accuracy: {:.3f}%'.format(accuracy_score(y_test.values, classifier.predict(X_test))*100)
    print '\nTraining set F1 score: {:.3f}%'.format(f1_score(y_train.values, classifier.predict(X_train))*100)
    print 'Test set F1 score: {:.3f}%\n'.format(f1_score(y_test.values, classifier.predict(X_test))*100)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=None, splitter='best')

Training set accuracy: 99.995%
Test set accuracy: 85.255%

Training set F1 score: 99.995%
Test set F1 score: 83.582%

GaussianNB()

Training set accuracy: 68.676%
Test set accuracy: 68.167%

Training set F1 score: 62.523%
Test set F1 score: 61.872%

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

Training set accuracy: 99.147%
Test set accuracy: 86.388%

Training set F1 score: 99.042%
Test set F1 score: 83.781%


### Choosing a model
After looking at the results, we have determined the **Decision Tree Classifier** to more accurately classify clients that have churned.

First, we instance the classifier with a Gini impurity criterion. This criterion affects the way the Decision Tree branches out. Gini impurity is a measure of how often a randomly chosen element from the set would be incorrectly labeled if it was randomly labeled according to the distribution of labels in the subset. It can be computed by summing the probability $f_i$ of an item with label $i$ being chosen times the probability $1-f_{i}$ of a mistake in categorizing that item. It reaches its minimum (0) when all cases in the node fall into a single target category. This criterion is useful for this kind of data because it deals better with continuous values than information gain or entropy.

Next, we utilize a maximum depth of 8 meaning the Decision Tree will reach 8 branch levels downwards at the most. This parameter is very important for this model, given the large amount of features and observations, it allows to reduce the specificity of the rules we find.

In [25]:
DTC = DecisionTreeClassifier(criterion='gini', max_depth=8).fit(X_train, y_train)

We score our model's accuracy and F1-measure both on the test set and the training set. Testing the model over the test set allows us to see how well our model performs on **different data** than the one that was used for training. Testing over the training set allows us to see how well we are performing over the **same data** that we used to train it.

The value of testing over both sets is to see **how different our results are**. This can give us some insight over how representative was the sampling method used, and if the model underfits or overfits the data.

In [26]:
accuracy_test = accuracy_score(y_test, DTC.predict(X_test)) * 100
accuracy_training = accuracy_score(y_train, DTC.predict(X_train)) * 100
print 'Test set accuracy: {:.3f}%'.format(accuracy_test)
print 'Training set accuracy: {:.3f}%'.format(accuracy_training)
print 'Accuracy differential: {:.3f}pp'.format(abs(accuracy_test - accuracy_training))

f1_test = f1_score(y_test, DTC.predict(X_test)) * 100
f1_training = f1_score(y_train, DTC.predict(X_train)) * 100
print '\nTest set F1 score: {:.3f}%'.format(f1_test)
print 'Training set F1 score: {:.3f}%'.format(f1_training)
print 'F1 score differential: {:.3f}pp'.format(abs(f1_test - f1_training))

Test set accuracy: 82.204%
Training set accuracy: 82.574%
Accuracy differential: 0.371pp

Test set F1 score: 78.684%
Training set F1 score: 79.140%
F1 score differential: 0.456pp


Upon agreeing on the similarities between the training and test sets, we set to evaluate the relationship between precision and recall through a confusion matrix. In this graph, we can tell how many true/false positives and true/false negatives we have obtained through our model.

In [ ]:
%matplotlib inline
cm = confusion_matrix(y_test, DTC.predict(X_test))
cm = cm.astype('float') / cm.sum(axis=1)[:, numpy.newaxis]

precision = float(cm[0][0]) / (cm[0][0] + cm[0][1]) * 100
recall = float(cm[0][0]) / (cm[0][0] + cm[1][0]) * 100

numpy.set_printoptions(precision=2)
pyplot.figure()
pyplot.imshow(cm, interpolation='nearest', cmap=pyplot.cm.Blues)
pyplot.title('Confusion matrix')
pyplot.colorbar()
tick_marks = numpy.arange(len(class_names))
pyplot.xticks(tick_marks, class_names, rotation=45)
pyplot.yticks(tick_marks, class_names)
pyplot.tight_layout()
pyplot.ylabel('TRUE')
pyplot.xlabel('PREDICTED')
cm = pandas.DataFrame(cm)

pyplot.show()

print cm
print '\nPrecision: {:.2f}%'.format(precision)
print 'Recall: {:.2f}%'.format(recall)

The results tell us that we can predict equally as well for clients that will stay and those that will churn.

We can now output our model to a PDF file:

In [ ]:
dot_data = StringIO()
export_graphviz(DTC, out_file=dot_data, feature_names=feature_cols, class_names=class_names, filled=True)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
graph.write_pdf("model.pdf")

We can also plot the top 10 most important features and their contribution towards reducing uncertainty through Gini impurity reduction.

In [ ]:
number_of_features = 10
importances = DTC.feature_importances_
std = numpy.std(DTC.feature_importances_,axis=0)
indices = numpy.argsort(importances)[::-1]

print '\nTop 10 features based on Gini impurity:'
for i in range(number_of_features):
    print '{}. {} (#{}): {:.3f}'.format(i + 1, feature_cols[i-1], indices[i], importances[indices[i]])

pyplot.figure()
pyplot.title("Feature importances")
pyplot.bar(range(number_of_features), importances[indices[:number_of_features]], color="gray", align="center")
pyplot.xticks(range(number_of_features), indices)
pyplot.xlim([-1, number_of_features])
pyplot.show()

In [ ]:
rules = data[
    ((data['NSE_E'] == 1) | (data['NSE_nan'] == 1))
    & (data['CAMBIO_PRIMA_UN PERIODO'] == 1)
    & (data['SUMA_ASEGURADA_MEDIA'] < (data['SUMA_ASEGURADA_MEDIA'].mean()+10))
    & (data['CANAL_NO_TRADICIONAL'] == 1)
    & (data['CORREDOR_MARSH'] == 0)
    & (data['CORREDOR_AMERICA_BROKERS'] == 0)
    & (data['CORREDOR_CONSEJEROS_CORREDORES_SEGUROS'] == 0)
    & (data['CORREDOR_CORREDORES_SEGUROS_FALABELLA'] == 0)
    & ((data['CORREDOR_MARIATEGUI'] == 1) | (data['CORREDOR_LA_UNION'] == 1))
    & (data['PRIMA_FINAL'] < (data['PRIMA_FINAL'].mean()-72))
]

rule_matches = len(rules)
churn_matches = len(rules.loc[rules['ABANDONO_A'] == 1])

if not(rules.empty):
    print 'Clients found by rule: {}'.format(rule_matches)
    print 'Clients that churned: {}'.format(churn_matches)
    print 'Rule accuracy: {:.3f}%'.format(churn_matches / float(rule_matches) * 100)
else:
    print 'No rules found'

In [ ]:
NSE = [
    'NSE_A',
    'NSE_A1',
    'NSE_A2',
    'NSE_B1',
    'NSE_B2',
    'NSE_C1',
    'NSE_C2',
    'NSE_D1',
    'NSE_D2',
    'NSE_E',
    'NSE_nan',
]

corredores = [
    'CORREDOR_MARSH', 
    'CORREDOR_CONSEJEROS_CORREDORES_SEGUROS', 
    'CORREDOR_CORREDORES_SEGUROS_FALABELLA',
    'CORREDOR_PRIMERA_CORREDORES_SEGURO', 
    'CORREDOR_MARIATEGUI', 
    'CORREDOR_LA_UNION', 
    'CORREDOR_CARLOS_UGAZ', 
    'CORREDOR_GERENCIA_RIESGOS', 
    'CORREDOR_GABEL', 
    'CORREDOR_CONTACTO_CORREDORES', 
    'CORREDOR_MN_ASOC', 
    'CORREDOR_AMERICA_BROKERS', 
    'CORREDOR_FBA_CORREDORES', 
    'CORREDOR_OTROS'
]

canales = [
    'CANAL_CORREDOR', 
    'CANAL_NO_TRADICIONAL', 
    'CANAL_DIRECTO', 
    'CANAL_FUERZA_VENTAS',
    'CANAL_NO_DETERMINADO'
]

riesgos = [
    'RIESGOEPSYSCTRSALUD', 
    'RIESGORIESGOSGENERALES', 
    'RIESGOSALUD', 
    'RIESGOVEHICULOSYSOAT', 
    'RIESGOVIDA'
]

In [ ]:
def test_pattern(pattern):
    for i in range(len(pattern)):
        if (i == 0):
            rules = (data[pattern[i]] == 1)
        else:
            if not(i+1 == len(pattern)):
                rules = rules | (data[pattern[i+1]] == 1)

    rule_matches = pandas.DataFrame(data[rules])
    match_count = len(rule_matches)
    churn_matches = len(rule_matches.loc[rule_matches['ABANDONO_A'] == 1])

    if not(rule_matches.empty):
        print 'Clients found by rule: {}'.format(match_count)
        print 'Clients that churned: {}'.format(churn_matches)
        print 'Rule accuracy: {:.3f}%'.format(churn_matches / float(match_count) * 100)
    else:
        print 'No rules found'

    baseline = churn_matches / float(match_count)

    for i in range(len(pattern)):
        rules = data[(data[pattern[i]] == 1)]
        rule_matches = len(rules)
        churn_matches = len(rules.loc[rules['ABANDONO_A'] == 1])

        if not(rules.empty):
            print '\n' + pattern[i]
            print 'Clients found by rule: {}'.format(rule_matches)
            print 'Clients that churned: {}'.format(churn_matches)
            print 'Rule accuracy: {:.3f}%'.format(churn_matches / float(rule_matches) * 100)
            print 'Churn rate change: {:.3f}pp'.format((churn_matches / float(rule_matches) - baseline) * 100)
        else:
            print 'No rules found'

In [ ]:
test_pattern(riesgos)

In [ ]:


for i in range(len(corredores)):
    if (i == 0):
        rules = (data[corredores[i]] == 1)
    else:
        if not(i+1 == len(corredores)):
            rules = rules | (data[corredores[i+1]] == 1)
            
rule_matches = pandas.DataFrame(data[rules])
match_count = len(rule_matches)
churn_matches = len(rule_matches.loc[rule_matches['ABANDONO_A'] == 1])

if not(rule_matches.empty):
    print 'Clients found by rule: {}'.format(match_count)
    print 'Clients that churned: {}'.format(churn_matches)
    print 'Rule accuracy: {:.3f}%'.format(churn_matches / float(match_count) * 100)
else:
    print 'No rules found'
    
baseline = churn_matches / float(match_count)

for i in range(len(corredores)):
    rules = data[(data[corredores[i]] == 1)]
    rule_matches = len(rules)
    churn_matches = len(rules.loc[rules['ABANDONO_A'] == 1])

    if not(rules.empty):
        print '\n' + corredores[i]
        print 'Clients found by rule: {}'.format(rule_matches)
        print 'Clients that churned: {}'.format(churn_matches)
        print 'Rule accuracy: {:.3f}%'.format(churn_matches / float(rule_matches) * 100)
        print 'Churn rate change: {:.3f}pp'.format((churn_matches / float(rule_matches) - baseline) * 100)
    else:
        print 'No rules found'

In [ ]:
len(data[(data['CORREDOR_CARLOS_UGAZ'] == 1)])

### Survival Analysis

In [ ]:
print list(churn_data_1.columns)

In [ ]:
warnings.filterwarnings('ignore')
from lifelines import KaplanMeierFitter
T = churn_data_1['tiempo'][churn_data_1['tiempo']<2000]
E = churn_data_1['ABANDONO_A'][churn_data_1['tiempo']<2000]
kmf = KaplanMeierFitter()
kmf.fit(T, event_observed=E) # more succiently, kmf.fit(T,E)

p = kmf.plot(ci_force_lines=True, title='Survival Analysis')

# Collect the plot object
kmf = plt.gcf() 

def pyplot(fig, ci=True, legend=True):
    # Convert mpl fig obj to plotly fig obj, resize to plotly's default
    py_fig = tls.mpl_to_plotly(fig, resize=True)
    
    # Add fill property to lower limit line
    if ci == True:
        style1 = dict(fill='tonexty')
        # apply style
        py_fig['data'][2].update(style1)
        
        # Change color scheme to black
        py_fig['data'].update(dict(line=Line(color='blue')))
    
    # change the default line type to 'step'
    py_fig['data'].update(dict(line=Line(shape='hv')))
    # Delete misplaced legend annotations 
    py_fig['layout'].pop('annotations', None)
    
    if legend == True:
        # Add legend, place it at the top right corner of the plot
        py_fig['layout'].update(
            showlegend=True,
            legend=Legend(
                x=1.05,
                y=1
            )
        )
        
    # Send updated figure object to Plotly, show result in notebook
    return py.iplot(py_fig)

pyplot(kmf, legend=False)

In [ ]:
from lifelines import CoxPHFitter

data_1 = churn_data_1[['tiempo','ABANDONO_A','NSE_A', 'NSE_A1', 'NSE_A2', 'NSE_B1', 'NSE_B2', 'NSE_C1', 'NSE_C2', 'NSE_D1', \
'NSE_D2', 'NSE_E','n_TIEMPO_FABRICACION','n_EDAD','CANAL_CORREDOR', 'CANAL_NO_TRADICIONAL', 'CANAL_DIRECTO', \
'n_NUMERO_SINIESTROS_TOTAL', 'n_NUMERO_SINIESTROS_FINAL', 'n_NUM_RIESGOS', 'n_NUM_PRODUCTOS',\
'TIPO_VEHICULO_AUTO', 'TIPO_VEHICULO_MOTO', 'TIPO_VEHICULO_RURAL','CORREDOR_MARSH', \
'CORREDOR_CONSEJEROS_CORREDORES_SEGUROS', 'CORREDOR_CORREDORES_SEGUROS_FALABELLA', 'CORREDOR_PRIMERA_CORREDORES_SEGURO', \
'CORREDOR_MARIATEGUI', 'CORREDOR_LA_UNION', 'CORREDOR_CARLOS_UGAZ', \
'CORREDOR_AMERICA_BROKERS', 'CORREDOR_FBA_CORREDORES', 'CORREDOR_OTROS']]

cf = CoxPHFitter()
cf.fit(data_1, 'tiempo', event_col= 'ABANDONO_A')

cf.print_summary()  # access the results using cf.summary

In [ ]:
churn_data_2 = churn_data_1[churn_data_1['tiempo']<2000]

kmf = KaplanMeierFitter()
ax = plt.subplot(111)

T = churn_data_2['tiempo']
C = churn_data_2['ABANDONO_A']
kmf.fit(T, event_observed=C, label=['BASELINE'])
kmf.survival_function_.plot(ax=ax)

lista_col = ['CORREDOR_MARSH', 'CORREDOR_CONSEJEROS_CORREDORES_SEGUROS', 'CORREDOR_CORREDORES_SEGUROS_FALABELLA', \
             'CORREDOR_PRIMERA_CORREDORES_SEGURO', 'CORREDOR_MARIATEGUI', 'CORREDOR_LA_UNION', 'CORREDOR_CARLOS_UGAZ', \
'CORREDOR_AMERICA_BROKERS', 'CORREDOR_FBA_CORREDORES', 'CORREDOR_OTROS', 'CORREDOR_GABEL']

for col in lista_col:
    print col + ': ' + str(len(churn_data_2[churn_data_2[col]==1])) + ' (churn: ' \
    + str(len(churn_data_2[(churn_data_2[col]==1) & (churn_data_2['ABANDONO_A']==1)])) + ')'
    f2 = churn_data_2[col]==1
    T2 = churn_data_2[f2]['tiempo']
    C2 = churn_data_2[f2]['ABANDONO_A']
    kmf.fit(T2, event_observed=C2, label=[col])
    kmf.survival_function_.plot(ax=ax)

plt.title('Survival Analysis')
kmf2 = plt.gcf()
pyplot(kmf2, ci=False)